In [ ]:
# Overall Notes:
# The code and exploration output is in seperate notebooks called WS (workshop), PLOT, CLEAN, and EDA 
# Did not have time to organize the notebooks so they are lots of errors, trials, etc.

# Process Notes:
# I spent somewhere between 30 and 40 hours on this and will never approach data in the same way (hugely positive)
# First, I followed the outlined flow but once I was into EDA I needed to figure out how to makes things work before I could determine and practice my EDA process.        
# I created many tables and copied them to OneNote where they become individual items and I can slide them around and compare. Did this mainly to see relationships
# Ran correlations and many different plot types

# Takeaways:
# Determining how to plot against a specific date range was my greatest challenge
# I need to practice, practice, practice.  Just scratching the surface but beginning to see the awesome power of pandas.
# Please let me know if you have any questions or would like to the lines of code I used for the analysis.  (started as hundreds but I removed a lot of the error items)
# Can't wait to do more

# Project 1 - Iowa Liquor 

You are a data scientist in residence at the Iowa State tax board. The Iowa State legislature is considering changes in the liquor tax rates and wants a report of current liquor sales by county and projections for the rest of the year. 

Your task is as follows:

* Calculate the yearly liquor sales for each store using the provided data. You can add up the transactions for each year, and store sales in 2015 specifically will be used later as your target variable.
* Use the data from 2015 to make a linear model using as many variables as you find useful to predict the yearly sales of all stores. You must use the sales from Jan to March as one of your variables.
* Use your model for 2015 to estimate total sales in 2016, extrapolating from the sales so far for Jan-March of 2016.
* Report your findings, including any projected increase or decrease in total sales (over the entire state) for the tax committee of the Iowa legislature.
* Use cross-validation to check how your model predicts to held out data compared to the model metrics on the full dataset.
* Fit your model(s) using one or both of the regularization tactics covered. Explain whether the regularized or the non-regularized model performed better and what the selected regression(s) are doing.



# Part 1

### Data Cleaning and EDA

In Part 1 of this two-part project, you will apply the skills you have learned manipulating data in Python with Pandas, Numpy, Matplotlib, Seaborn and other tools to import the Iowa Liquor data, clean the dataset, then perform exploratory analysis using visual and statistical methods.

### Requirements:

**Identify the problem**
- Write a high quality problem statement
- Describe the goals of your study and criteria for success

**Acquire and clean the data**
- Verify the dataset is in the 'Assets' folder of this project - the data is from [Iowa.gov](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy), filtered and
reduced
- Import data using the Pandas Library
- Format, clean, slice, and combine the data in Python

**Explore the data**
- Perform exploratory analysis methods with visualization and statistical analysis
- Determine outliers, skew distribution of important variables (if any)
- Determine correlations / causations in the data
- State the risks and assumptions of your data
- Identify 5 relationships, trends, or other intersting attributes of the data set

# Identify the Problem

The goal of this two-part project is to build a model predicting future sales. Write a problem statement and identify SMART goals 

### Problem Statement

_write your problem statement here_

Determine the types of liquor

In [ ]:
# The Iowa State legislature will analyze the liquor sale data in order to provide additional in making the decision to raise alchohol tax rates.       

### SMART Goals

_write your SMART goals here_



In [ ]:
# Review the liquar data set to determine what relations can be found that would lend to prediction criteria

# Download Data



If you navigate to the [data.iowa.gov](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy) page for the liquor sales data and click "download" - you'll end up with a 3.4GB/12.6 million row file containing all transactions by product for every class-E liquor store in the state since January 1, 2012

For this project, we are providing a trimmed version of this dataset which contains a sample of the data. If you would like to try using a larger sample of the data, reach out to John or Joseph


** The folowing code verifies that the dataset is in the right location **

If the code below returns 'false' - reach out to your instructors.

In [2]:
# Project due Monday Nov, 20th

import os
os.path.isfile('C:/Users/cmagann/Desktop/GA_DS/Homework/Iowa Project/Iowa_Liquor_sample.csv') 

#slice, aggregate, etc. for items of interest

True

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries import converter as pdtc
import matplotlib.units as munits
import numpy as np
munits.registry[np.datetime64] = pdtc.DatetimeConverter()
import seaborn as sns
from scipy import stats

# Make plots larger
plt.rcParams['figure.figsize'] = (10, 6)
%matplotlib inline

In [4]:
lq = pd.read_csv('C:/Users/cmagann/Desktop/GA_DS/Homework/Iowa Project/Iowa_Liquor_sample.csv')

In [ ]:
# Option 1: adjust pd.read_csv
lq = pd.read_csv('Iowa_Liquor_sample.csv',parse_dates=['Date'],infer_datetime_format=True)

In [ ]:
# Option 2: convert the 'Date' column
lq['Date'] = pd.to_datetime(lq['Date'],errors = 'coerce',infer_datetime_format = True)

# Load Data and Clean

Start by loading the data with pandas. You may need to parse the date columns appropriately.

### Explore the head and tail

View the head and tail of the data set; take a look at the columns. 

Can you identify what each of the columns are describing? 

How many rows / columns are there in the dataset?

In [5]:
lq.head()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,10/1/2015,2633,DES MOINES,50320,77.0,Polk,1011300.0,TENNESSEE WHISKIES,85,26827,Jack Daniels Old #7 Black Lbl,1000,$18.38,$27.57,1320,"$36,392.40",1320.0,348.71
1,1/15/2016,3814,WEST DES MOINES,50266,25.0,Dallas,1031080.0,VODKA 80 PROOF,301,38177,Titos Handmade Vodka,1000,$11.92,$17.88,1800,"$32,184.00",1800.0,475.51
2,11/18/2015,4677,CORALVILLE,52241,52.0,Johnson,1032080.0,IMPORTED VODKA,35,34425,Grey Goose Vodka,1750,$35.42,$53.13,552,"$29,327.76",966.0,255.19
3,10/12/2015,3814,WEST DES MOINES,50266,25.0,Dallas,1011500.0,STRAIGHT RYE WHISKIES,255,27102,Templeton Rye,750,$18.09,$27.14,960,"$26,054.40",720.0,190.20
4,5/4/2015,3814,WEST DES MOINES,50266,25.0,Dallas,1012100.0,CANADIAN WHISKIES,115,11788,Black Velvet,1750,$9.70,$14.93,1200,"$17,916.00",2100.0,554.76


In [6]:
lq.tail()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
270950,3/26/2015,3920,CLINTON,52732,23.0,Clinton,1041100.0,AMERICAN DRY GINS,370,32232,Seagrams Extra Dry Gin,100,$0.97,$1.46,1,$1.46,0.1,0.03
270951,3/19/2015,4889,DES MOINES,50315,77.0,Polk,1041100.0,AMERICAN DRY GINS,370,32232,Seagrams Extra Dry Gin,100,$0.97,$1.46,1,$1.46,0.1,0.03
270952,7/15/2015,3650,HOLSTEIN,51025,47.0,Ida,1062200.0,PUERTO RICO & VIRGIN ISLANDS RUM,55,44215,Barton Light Rum,100,$0.89,$1.34,1,$1.34,0.1,0.03
270953,6/11/2015,2587,JOHNSTON,50131,77.0,Polk,1062200.0,PUERTO RICO & VIRGIN ISLANDS RUM,55,44215,Barton Light Rum,100,$0.89,$1.34,1,$1.34,0.1,0.03
270954,6/11/2015,2587,JOHNSTON,50131,77.0,Polk,1081600.0,WHISKEY LIQUEUR,421,64870,Fireball Cinnamon Whisky,100,$0.89,$1.34,1,$1.34,0.1,0.03


In [29]:
lq.shape

(270955, 18)

In [ ]:
# remove spaces
lq.columns = lq.columns.str.replace(' ','')

In [ ]:
# remove both left and right parentheses
lq.columns = lq.columns.str.replace('(','')
lq.columns = lq.columns.str.replace(')','')
lq.columns.values

In [ ]:
lq['StateBottleCost'] = lq['StateBottleCost'].apply(lambda x: x.replace('$',''))
lq['StateBottleRetail'] = lq['StateBottleRetail'].apply(lambda x: x.replace('$',''))
lq['SaleDollars'] = lq['SaleDollars'].apply(lambda x: x.replace('$',''))

lq['StateBottleCost'] = pd.to_numeric(lq['StateBottleCost'],errors='coerce')
lq['StateBottleRetail'] = pd.to_numeric(lq['StateBottleRetail'],errors='coerce')
lq['SaleDollars'] = pd.to_numeric(lq['SaleDollars'],errors='coerce')

### Parse dates

Using '.dtypes' on our dataframe allows us to view the data type of each of our collumns. Pandas does its best to infer data types on ingest, but we may still need to make assumptions. [Pandas Dtype Basics](https://pandas.pydata.org/pandas-docs/stable/basics.html#basics-dtypes)

In [30]:
# use .dtypes on the liquor dataframe to view the data type of each column
lq.dtypes

Date                      object
Store Number               int64
City                      object
Zip Code                  object
County Number            float64
County                    object
Category                 float64
Category Name             object
Vendor Number              int64
Item Number                int64
Item Description          object
Bottle Volume (ml)         int64
State Bottle Cost         object
State Bottle Retail       object
Bottles Sold               int64
Sale (Dollars)            object
Volume Sold (Liters)     float64
Volume Sold (Gallons)    float64
dtype: object

Note that the 'Date' column has the dtype 'object' - this is the pandas data type designation for a string or a data type it doesn't recognise. 

We want our 'Date' column to be interpreted as datatime by pandas so we can perform time-based grouping and other functions on this column, so we have to convert it to a datetime datatype.

Pandas gives us some options:
- Adjust our pd.read_csv to infer datetimes on import
- Directly convert the date column

[Pandas pd.read_csv datetime handling documentation](https://pandas.pydata.org/pandas-docs/stable/io.html#datetime-handling)

[Pandas pd.to_datetime documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)

In [5]:
# Option 1: adjust pd.read_csv
lq = pd.read_csv('Iowa_Liquor_sample.csv',parse_dates=['Date'],infer_datetime_format=True)

In [6]:
# Option 2: convert the 'Date' column
lq['Date'] = pd.to_datetime(lq['Date'],errors = 'coerce',infer_datetime_format = True)

In [150]:
# check .dtypes again
lq.dtypes

Date                     datetime64[ns]
Store Number                      int64
City                             object
Zip Code                         object
County Number                   float64
County                           object
Category                        float64
Category Name                    object
Vendor Number                     int64
Item Number                       int64
Item Description                 object
Bottle Volume (ml)                int64
State Bottle Cost                object
State Bottle Retail              object
Bottles Sold                      int64
Sale (Dollars)                   object
Volume Sold (Liters)            float64
Volume Sold (Gallons)           float64
dtype: object

### Clean Column Names

Are there spaces in the column names? When columns have spaces in there name, it makes it difficult to use pandas in the form of df.column.method() (ie, liquor.City.value_counts)

For a multi-word column name, you would have to use df['Zip Code'] instead of df.ZipCode

Additionally, some of our columns have parantheses in their names. 

We can remove spaces and special characters from our columns, which will help keep our code clean.

In [100]:
# view  column names
lq.columns.values

array(['Date', 'Store Number', 'City', 'Zip Code', 'County Number',
       'County', 'Category', 'Category Name', 'Vendor Number',
       'Item Number', 'Item Description', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'], dtype=object)

In [7]:
# remove spaces
lq.columns = lq.columns.str.replace(' ','')

In [8]:
# remove both left and right parentheses
lq.columns = lq.columns.str.replace('(','')
lq.columns = lq.columns.str.replace(')','')
lq.columns.values

array(['Date', 'StoreNumber', 'City', 'ZipCode', 'CountyNumber', 'County',
       'Category', 'CategoryName', 'VendorNumber', 'ItemNumber',
       'ItemDescription', 'BottleVolumeml', 'StateBottleCost',
       'StateBottleRetail', 'BottlesSold', 'SaleDollars',
       'VolumeSoldLiters', 'VolumeSoldGallons'], dtype=object)

In [9]:
lq['StateBottleCost'] = lq['StateBottleCost'].apply(lambda x: x.replace('$',''))
lq['StateBottleRetail'] = lq['StateBottleRetail'].apply(lambda x: x.replace('$',''))
lq['SaleDollars'] = lq['SaleDollars'].apply(lambda x: x.replace('$',''))

lq['StateBottleCost'] = pd.to_numeric(lq['StateBottleCost'],errors='coerce')
lq['StateBottleRetail'] = pd.to_numeric(lq['StateBottleRetail'],errors='coerce')
lq['SaleDollars'] = pd.to_numeric(lq['SaleDollars'],errors='coerce')

In [40]:
lq.head()

,Date,StoreNumber,City,ZipCode,CountyNumber,County,Category,CategoryName,VendorNumber,ItemNumber,ItemDescription,BottleVolumeml,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons
0,2015-10-01,2633,DES MOINES,50320,77.0,Polk,1011300.0,TENNESSEE WHISKIES,85,26827,Jack Daniels Old #7 Black Lbl,1000,18.38,27.57,1320,NaN,1320.0,348.71
1,2016-01-15,3814,WEST DES MOINES,50266,25.0,Dallas,1031080.0,VODKA 80 PROOF,301,38177,Titos Handmade Vodka,1000,11.92,17.88,1800,NaN,1800.0,475.51
2,2015-11-18,4677,CORALVILLE,52241,52.0,Johnson,1032080.0,IMPORTED VODKA,35,34425,Grey Goose Vodka,1750,35.42,53.13,552,NaN,966.0,255.19
3,2015-10-12,3814,WEST DES MOINES,50266,25.0,Dallas,1011500.0,STRAIGHT RYE WHISKIES,255,27102,Templeton Rye,750,18.09,27.14,960,NaN,720.0,190.20
4,2015-05-04,3814,WEST DES MOINES,50266,25.0,Dallas,1012100.0,CANADIAN WHISKIES,115,11788,Black Velvet,1750,9.70,14.93,1200,NaN,2100.0,554.76


### Clean Numerics

Look at the .dtypes and the .head() of the dataframe again - are there any columns that should be a numeric data type (float, int, etc) that are still objects? (hint - follow the $money)

[Pandas Series.replace()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.replace.html)

[Pandas pd.to_numeric()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html)

In [11]:
lq[('CountyNumber')] = lq[('CountyNumber')].astype(int)

In [12]:
lq[('Category')] = lq[('Category')].astype(int)

In [ ]:
lq[('SaleDollars')] = lq[('SaleDollars')].astype(int)

In [ ]:
# convert string/object columns to numeric
lq['CountyNumber'] = pd.to_numeric(lq['CountyNumber'], errors='coerce')

In [51]:
lq.dtypes

Date                 datetime64[ns]
StoreNumber                   int64
City                         object
ZipCode                      object
CountyNumber                  int32
County                       object
Category                      int32
CategoryName                 object
VendorNumber                  int64
ItemNumber                    int64
ItemDescription              object
BottleVolumeml                int64
StateBottleCost             float64
StateBottleRetail           float64
BottlesSold                   int64
SaleDollars                 float64
VolumeSoldLiters            float64
VolumeSoldGallons           float64
dtype: object

In [44]:
# examine your .dtypes and .head() to confirm your type adjustements
lq.head()

,Date,StoreNumber,City,ZipCode,CountyNumber,County,Category,CategoryName,VendorNumber,ItemNumber,ItemDescription,BottleVolumeml,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons
0,2015-10-01,2633,DES MOINES,50320,77.0,Polk,1011300.0,TENNESSEE WHISKIES,85,26827,Jack Daniels Old #7 Black Lbl,1000,18.38,27.57,1320,NaN,1320.0,348.71
1,2016-01-15,3814,WEST DES MOINES,50266,25.0,Dallas,1031080.0,VODKA 80 PROOF,301,38177,Titos Handmade Vodka,1000,11.92,17.88,1800,NaN,1800.0,475.51
2,2015-11-18,4677,CORALVILLE,52241,52.0,Johnson,1032080.0,IMPORTED VODKA,35,34425,Grey Goose Vodka,1750,35.42,53.13,552,NaN,966.0,255.19
3,2015-10-12,3814,WEST DES MOINES,50266,25.0,Dallas,1011500.0,STRAIGHT RYE WHISKIES,255,27102,Templeton Rye,750,18.09,27.14,960,NaN,720.0,190.20
4,2015-05-04,3814,WEST DES MOINES,50266,25.0,Dallas,1012100.0,CANADIAN WHISKIES,115,11788,Black Velvet,1750,9.70,14.93,1200,NaN,2100.0,554.76


### Null Values

Evaluate the null values in the dataset. 

Questions to guide your process:
- What columns have null values?
- How many nulls values are in each column?
- Will the missing values effect your analysis? 
- Can you afford to remove (drop) the null values from your dataset?

[df.isnull()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.isnull.html)

[df.dropna()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html)

[O'Reilly article on dealing with nulls using pandas](https://www.oreilly.com/learning/handling-missing-data)

[Data School video on missing values in pandas](https://www.youtube.com/watch?v=fCMrO_VzeL8)

In [101]:
lq.isnull().sum()

Date                    0
StoreNumber             0
City                    0
ZipCode                 0
CountyNumber         1077
County               1077
Category               68
CategoryName          632
VendorNumber            0
ItemNumber              0
ItemDescription         0
BottleVolumeml          0
StateBottleCost         0
StateBottleRetail       0
BottlesSold             0
SaleDollars          2728
VolumeSoldLiters        0
VolumeSoldGallons       0
dtype: int64

In [139]:
#lq['County'].value_counts()
lq[lq.County.isnull()]

,Date,StoreNumber,City,ZipCode,CountyNumber,County,Category,CategoryName,VendorNumber,ItemNumber,ItemDescription,BottleVolumeml,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons
1393,2016-03-16,3820,SIOUX CITY,51103,NaN,NaN,1012100.0,CANADIAN WHISKIES,260,11297,Crown Royal Canadian Whisky,1000,18.75,28.13,60,NaN,60.00,15.85
1394,2016-03-16,2238,ALTOONA,50009,NaN,NaN,1012100.0,CANADIAN WHISKIES,260,11297,Crown Royal Canadian Whisky,1000,18.75,28.13,60,NaN,60.00,15.85
1457,2016-03-09,3820,SIOUX CITY,51103,NaN,NaN,1011300.0,TENNESSEE WHISKIES,85,26827,Jack Daniels Old #7 Black Lbl,1000,18.38,27.57,60,NaN,60.00,15.85
1458,2016-03-02,3820,SIOUX CITY,51103,NaN,NaN,1011300.0,TENNESSEE WHISKIES,85,26827,Jack Daniels Old #7 Black Lbl,1000,18.38,27.57,60,NaN,60.00,15.85
1588,2016-02-22,2460,HAMPTON,50441,NaN,NaN,1012100.0,CANADIAN WHISKIES,55,12408,Canadian Ltd Whisky,1750,7.94,12.51,126,NaN,220.50,58.25
1631,2016-03-09,2637,DAVENPORT,52804,NaN,NaN,1082900.0,MISC. IMPORTED CORDIALS & LIQUEURS,192,65256,Jagermeister Liqueur,750,12.05,18.08,84,NaN,63.00,16.64
2236,2016-03-02,5222,CEDAR RAPIDS,52402,NaN,NaN,1052010.0,IMPORTED GRAPE BRANDIES,389,49186,Remy Martin Vsop Cognac,750,21.98,32.97,36,NaN,27.00,7.13
2632,2016-03-30,5222,CEDAR RAPIDS,52402,NaN,NaN,1052010.0,IMPORTED GRAPE BRANDIES,420,48106,Hennessy Vs Cognac,750,18.99,28.49,36,NaN,27.00,7.13
2634,2016-03-09,5222,CEDAR RAPIDS,52402,NaN,NaN,1052010.0,IMPORTED GRAPE BRANDIES,420,48106,Hennessy Vs Cognac,750,18.99,28.49,36,NaN,27.00,7.13
2637,2016-02-24,5222,CEDAR RAPIDS,52402,NaN,NaN,1052100.0,NaN,420,48106,Hennessy Vs Cognac,750,18.99,28.49,36,NaN,27.00,7.13


In [85]:
#lq.groupby('County').SaleDollars.describe()
#.sort('star_rating', ascending=False)
#iris.groupby('species').petal_area.describe().unstack()
lq.sort_values(by = 'SaleDollars', ascending=False)
#lq.sort_values(by = 'star_rating', ascending=False)

,Date,StoreNumber,City,ZipCode,CountyNumber,County,Category,CategoryName,VendorNumber,ItemNumber,ItemDescription,BottleVolumeml,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons
2728,2016-01-15,3385,CEDAR RAPIDS,52402,57.0,Linn,1031200.0,VODKA FLAVORED,260,77741,Smirnoff Raspberry,1000,9.25,13.88,72,999.36,72.00,19.02
2730,2015-11-03,3477,COUNCIL BLUFFS,51501,78.0,Pottawattamie,1051010.0,AMERICAN GRAPE BRANDIES,205,52599,E & J Vs Brandy Round,1000,6.94,10.41,96,999.36,96.00,25.36
2731,2015-10-01,2616,CLINTON,52732,23.0,Clinton,1031080.0,VODKA 80 PROOF,260,37997,Smirnoff Vodka 80 Prf,1000,9.25,13.88,72,999.36,72.00,19.02
2732,2015-08-13,4829,DES MOINES,50314,77.0,Polk,1031080.0,VODKA 80 PROOF,260,37997,Smirnoff Vodka 80 Prf,1000,9.25,13.88,72,999.36,72.00,19.02
2733,2015-03-18,3447,SIOUX CITY,51106,97.0,Woodbury,1051010.0,AMERICAN GRAPE BRANDIES,205,52599,E & J Vs Brandy Round,1000,6.94,10.41,96,999.36,96.00,25.36
2734,2015-02-26,2633,DES MOINES,50320,77.0,Polk,1031080.0,VODKA 80 PROOF,260,37997,Smirnoff Vodka 80 Prf,1000,9.25,13.88,72,999.36,72.00,19.02
2729,2015-12-22,3354,DAVENPORT,52807,82.0,Scott,1031080.0,VODKA 80 PROOF,260,37997,Smirnoff Vodka 80 Prf,1000,9.25,13.88,72,999.36,72.00,19.02
2757,2015-01-22,2595,DENISON,51442,24.0,Crawford,1012100.0,CANADIAN WHISKIES,260,10808,Crown Royal Regal Apple,1000,18.50,27.75,36,999.00,36.00,9.51
2752,2015-04-01,2512,IOWA CITY,52240,52.0,Johnson,1012100.0,CANADIAN WHISKIES,260,11297,Crown Royal Canadian Whisky,1000,18.50,27.75,36,999.00,36.00,9.51
2753,2015-03-03,3477,COUNCIL BLUFFS,51501,78.0,Pottawattamie,1012100.0,CANADIAN WHISKIES,260,11297,Crown Royal Canadian Whisky,1000,18.50,27.75,36,999.00,36.00,9.51


In [86]:
# lq.groupby('City').County.value_counts()
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
lq.groupby('County').City.value_counts()
# SIOUX CITY / 7909 instances
# CEDAR RAPIDS / 18778 instances

height has been deprecated.



County         City             
Adair          STUART                 285
               GREENFIELD             195
               FONTANELLE              62
               ADAIR                   43
Adams          CORNING                234
Allamakee      WAUKON                 800
               LANSING                151
               HARPERS FERRY           48
               POSTVILLE               46
Appanoose      CENTERVILLE            976
               MORAVIA                 67
Audubon        AUDUBON                205
               EXIRA                   22
Benton         BELLE PLAINE           414
               SHELLSBURG             330
               VINTON                 206
               URBANA                  29
Black Hawk     WATERLOO              8411
               CEDAR FALLS           5735
               EVANSDALE              484
               LA PORTE CITY          248
               RAYMOND                144
               HUDSON                  40
 

In [109]:
lq['City'].value_counts()

DES MOINES         23724
CEDAR RAPIDS       18888
DAVENPORT          11580
WATERLOO            8425
COUNCIL BLUFFS      8060
SIOUX CITY          7992
IOWA CITY           7951
AMES                7548
WEST DES MOINES     7162
DUBUQUE             6915
CEDAR FALLS         5735
ANKENY              4836
MASON CITY          4191
BETTENDORF          3709
CORALVILLE          3490
MUSCATINE           3397
BURLINGTON          3144
CLINTON             3111
FORT DODGE          2989
WINDSOR HEIGHTS     2811
MARSHALLTOWN        2694
NEWTON              2544
STORM LAKE          2533
MARION              2489
URBANDALE           2438
OTTUMWA             2295
JOHNSTON            2141
ALTOONA             2113
CLEAR LAKE          2083
SPENCER             1910
                   ...  
BUSSEY                16
DANVILLE              15
SCHALLER              15
VAN METER             15
GOLDFIELD             15
WASHBURN              15
ALTA                  14
WALL LAKE             14
DOWS                  13


In [98]:
lq['County'].value_counts()
# county with number of instances

Polk             49102
Linn             23511
Scott            16678
Black Hawk       15083
Johnson          13189
Pottawattamie     9111
Story             8959
Woodbury          8562
Dubuque           7757
Cerro Gordo       6374
Des Moines        4090
Muscatine         3983
Clinton           3576
Wapello           3529
Dickinson         3413
Lee               3324
Webster           3154
Marshall          2996
Jasper            2834
Buena Vista       2748
Dallas            2708
Marion            2603
Warren            2465
Bremer            2243
Boone             2110
Poweshiek         2089
Clay              1917
Carroll           1915
Jones             1874
O'Brien           1720
                 ...  
Greene             677
Wright             672
Shelby             663
Ida                634
Howard             607
Humboldt           588
Adair              585
Grundy             566
Pocahontas         525
Mills              508
Louisa             484
Lucas              476
Chickasaw  

In [54]:
lq.SaleDollars.mean()

103.13648264345822

In [55]:
lq.SaleDollars.describe()

count    268227.000000
mean        103.136483
std         112.321518
min           1.340000
25%          30.040000
50%          69.120000
75%         134.880000
max         999.360000
Name: SaleDollars, dtype: float64

In [6]:
# Remove rows containing any null values 
# Research indicates the null values are such a small percentage that the removal will be non impactful to the model
lq = lq.dropna(how='any')

In [88]:
lq[lq.County.isnull()]

,Date,StoreNumber,City,ZipCode,CountyNumber,County,Category,CategoryName,VendorNumber,ItemNumber,ItemDescription,BottleVolumeml,StateBottleCost,StateBottleRetail,BottlesSold,SaleDollars,VolumeSoldLiters,VolumeSoldGallons


In [75]:
lq.isnull().sum()

Date                 0
StoreNumber          0
City                 0
ZipCode              0
CountyNumber         0
County               0
Category             0
CategoryName         0
VendorNumber         0
ItemNumber           0
ItemDescription      0
BottleVolumeml       0
StateBottleCost      0
StateBottleRetail    0
BottlesSold          0
SaleDollars          0
VolumeSoldLiters     0
VolumeSoldGallons    0
dtype: int64

In [76]:
lq.SaleDollars.mean()

103.07033798031746

In [77]:
lq.SaleDollars.describe()

count    266554.000000
mean        103.070338
std         112.289438
min           1.340000
25%          30.020000
50%          69.120000
75%         134.880000
max         999.360000
Name: SaleDollars, dtype: float64

## Exploratory Data Analysis

Using pandas (and other tools such as NumPy, Matplotlib, or Seaborn) explore the Iowa Liquor dataset.

**Remember** Your goal in Part 2 will be to predict future sales. Look for relationships, trends, and features that may assist you in that task.

### Identify 5 trends, relationships, or things that stand out to you in the dataset

Display yout findings below

In [ ]:
# ** Example ** 

# Relationship between mean Bottles Sold and average SaleDollars (by store)

# There is a strong possitive relationship between the average number of bottles sold per transaction at a store and the average total sale amount per transaction at the same store. 

# Digging deeper, by comparing BottlesSold to StateBottleRetai we can see that there is almost no relationship between the average bottle price per transaction at a store and the average number of bottles sold per transaction; that is stores which are selling more bottles per transaction are not nescesarily selling cheaper items, and stores that are selling less bottles are not selling more expensive items.


In [ ]:
# **Example
# Relationship between mean Bottles Sold and average SaleDollars (by store)
# There is a strong possitive relationship between the average number of bottles sold per transaction at a store and the average total sale amount per transaction at the same store.
# Digging deeper, by comparing BottlesSold to StateBottleRetai we can see that there is almost no relationship between the average bottle price per transaction at a store and the average number of bottles sold per transaction; that is stores which are selling more bottles per transaction are not nescesarily selling cheaper items, and stores that are selling less bottles are not selling more expensive items.

### Finding #1

*description of your finding*

In [ ]:
# Individually, the less expensive items typically had the highest gross sales (bottles sold to sale dollars)

### Finding #2

*description of your finding*

In [ ]:
# Top sales numbers corrlate to population density and the top five grossing cities are all near major highways

### Finding #3

*description of your finding*

In [ ]:
# The highest grossing categories consisted of higher cost items

### Finding #4

*description of your finding*

In [ ]:
# The top items in the BottlesSold and SalesDollars do not equate evenly, indicating a correlation in profit margin

### Finding #5

*description of your finding*

In [ ]:
# Black Velvet has a great profit margin

In [ ]:
# Additional findings below:

In [ ]:
# Three of the top five highest grossing individual items are whiskey but the best selling category is vodka products.  
# Three of the top five product sale in volume of bottles sold are vodka products.

In [ ]:
# Three of the top five grossing stores are in Cedar Rapids

In [ ]:
# The city of DeMoine had the highest gross sales.  
# DeMoines' gross sales were 37% higher than the next highest grossing city, Cedar Rapids.

In [ ]:
# Outlier in scatter plot, going to do more research on a SaleDollars to VolumeSoldGallons comparison